# Разработка сентимент-анализа под задачу заказчика

### К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется.

### Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

### Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму.

In [1]:
import requests
import bs4
import codecs
import os
import pandas as pd
import numpy as np

In [12]:
with open('test_week_6.csv') as test_file:
    test_data = test_file.read()

In [15]:
print(test_data)

<review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.
</review>

т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.
работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас , но на солнце 

In [18]:
print('Количество отзывов, предоставленных заказчиком: {}'.format(test_data.count('<review>')))

Количество отзывов, предоставленных заказчиком: 100


### Видим, что в тестовой выборке 100 отзывов на телефоны на русском языке. Соберём обучающую выборку отзывов на телефоны с сайта https://torg.mail.ru/review/goods/mobilephones.

In [19]:
def load_user_data(page):
    '''
    функция, возвращающая html текс страницы сайта
    '''
    url = 'https://torg.mail.ru/review/goods/mobilephones/?page={}'.format(page)
    request = requests.get(url)
    return request.text

def contain_reviews_data(text):
    '''
    функция, возвращающая содержимое тега div с классом content_grid__center_column
    в этом тэге содержатся отзывы
    '''
    soup = bs4.BeautifulSoup(text, 'lxml')
    review_list = soup.find('div', {'class':'content_grid__center_column'})
    return review_list

Возьмём отзывы с 900 страниц. Их будет порядка 18000.

In [4]:
# Сохраняем содержимое тэгов, содержащих отзывы в файлы
for i in range(1, 901):
    data = load_user_data(i)
    if contain_reviews_data(data):
        with open('html/page_{}.html'.format(i), 'w') as output_file:
            output_file.write(data.encode('cp1251'))

In [20]:
import re
review_re = re.compile('review-item js-review_item js-ustat_block js-ustat_block_review*')

In [21]:
def parse_datafile(filename):
    '''
    Осуществляем парсинг сохранённых страниц и возвращаем кортеж списков текстов отзывов с оценками
    '''
    results = []
    with open(filename) as input_file:
        text = input_file.read()
    
    parser = bs4.BeautifulSoup(text, 'lxml')
    review_list = parser.findAll('div', attrs={'class':'review-item__body'})
    for r in review_list:
        if r.find('a', {'class':'more'}):
            review_text = r.find('a', {'class':'more'}).get('full-text').encode('utf-8')
        else:
            review_text = r.find('span', {'class':'js-more-text'}).text.encode('utf-8')
        
        review_rating = r.find('span', {'class':'review-item__rating-counter'}).text.encode('utf-8')
        
        results.append({'review_text': review_text, 'review_rating': review_rating})
    return results

In [23]:
# Первые 10 отзывов/оценок
for cnt, i in enumerate(parse_datafile('html/page_1.html'), 1):
    if cnt == 11:
        break
    print('{}) {}'.format(cnt, i['review_text']))
    print(i['review_rating'])
    print

1) Характеристики неплохие,качественными дисплей и камера. Я довольна, тем более , что это бюджетный вариант. Ещё бюджетные варианты описаны https://smartbobr.ru/dlyasmartfonov/smartfony-do-10000-rublej/#Xiaomi_Redmi_Note_5A__10000, где я его и выбрала												
4

2) тихий звонок и музыка - может после купания (прыгнул за ребенком в озеро с тел)<br />
раза 4 вис... пришлось отключать батарею... может вирусы пытались взять тел под управление а он не дался<br />
не могу очистить оперативную память, благо сам себя чистит и сообщает очистил мол 4.74 оперативной памяти
5

3) Один из лучших телефонов на настоящее время! Очень быстрый и стильный! Хорошая камера! А главное ,батарея очень долгая и конечно цена!												
5

4) Ужасная камера, отвратительная просто. Звук громкости, тихий очень. Подсветки на клавишах нет. Индикатора пропущенных вызовов нет. Не стоит таких денег, как на витрине. Хуже китайцев. А, но да за марку самсунг денег же содрать нужно. Фуфло полное. Кто думает его поку

In [24]:
# Получаем список кортежей, состоящих из отзывов и оценок
results = []
for filename in os.listdir('html/'):
    results.extend(parse_datafile('html/' + filename))

In [62]:
train_data = pd.DataFrame(results)

In [63]:
train_data.head()

,review_rating,review_text
0,4,"Характеристики неплохие,качественными дисплей ..."
1,5,тихий звонок и музыка - может после купания (п...
2,5,Один из лучших телефонов на настоящее время! О...
3,2,"Ужасная камера, отвратительная просто. Звук гр..."
4,1,Тормозит страшно. Самостоятельно обновляется п...


### Разметим обучающую выборку. Поскольку мы решаем задачу бинарной классификации и нас интересуют только 2 класса: положительные и отрицательные отзывы, преобразуем оценки отзывов в 0 и 1 описанным ниже способом. Отзывы с оценками 3, 3.5 скорее относятся к нейтральным отзывам, поэтому их исключим из дальнейшего рассмотрения.

In [64]:
def pos_neg(i):
    '''
    Преобразование оценочных показателей устройств в бинарные значения:
    1 - положительный отзыв, 0 - негативный отзыв
    '''
    if i in ['0', '1', '1,5', '2', '2,5']:
        return 0
    elif i in ['4', '4,5', '5']:
        return 1

In [65]:
train_data['labels'] = train_data.review_rating.apply(pos_neg)

In [66]:
train_data = train_data.drop(['review_rating'], axis = 1)
train_data = train_data.dropna()

In [67]:
train_data.head(15)

,review_text,labels
0,"Характеристики неплохие,качественными дисплей ...",1.0
1,тихий звонок и музыка - может после купания (п...,1.0
2,Один из лучших телефонов на настоящее время! О...,1.0
3,"Ужасная камера, отвратительная просто. Звук гр...",0.0
4,Тормозит страшно. Самостоятельно обновляется п...,0.0
5,Телефон прелесть! Качество отменное: ни одного...,1.0
6,"Очень хороший телефон, пользуюсь третий месяц....",1.0
7,Айфон спорный аппарат. Начну с недостатков. Ак...,1.0
8,"Пробыл 4 года,батарея стала держать максимум т...",1.0
9,За эти деньги лучше не найти!!!!!!Купил спонта...,1.0


In [68]:
print('Размер выборки: {}'.format(train_data.shape[0]))

Размер выборки: 15851


In [69]:
texts = list(train_data['review_text'])
labels = list(train_data['labels'])

### Предобработка текстов.

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.model_selection import RandomizedSearchCV
from nltk.corpus import stopwords
import string

In [71]:
# удаляем из текстов отзывов некоторые символы
texts = [i.replace('\n', ' ').replace('\r', '').replace('\t', ' ').replace('<br />', ' ').replace('<br>', ' ')
               if len(i.split()) > 1
               else i.replace('\n', ' ').replace('\r', '').replace('\t', ' ').replace('<br />', ' ').replace('<br>', ' ')
               for i in texts]

In [72]:
train_data = pd.DataFrame({'label': labels,
              'text' : texts,
             })
train_data.head(10)

,label,text
0,1.0,"Характеристики неплохие,качественными дисплей ..."
1,1.0,тихий звонок и музыка - может после купания (п...
2,1.0,Один из лучших телефонов на настоящее время! О...
3,0.0,"Ужасная камера, отвратительная просто. Звук гр..."
4,0.0,Тормозит страшно. Самостоятельно обновляется п...
5,1.0,Телефон прелесть! Качество отменное: ни одного...
6,1.0,"Очень хороший телефон, пользуюсь третий месяц...."
7,1.0,Айфон спорный аппарат. Начну с недостатков. Ак...
8,1.0,"Пробыл 4 года,батарея стала держать максимум т..."
9,1.0,За эти деньги лучше не найти!!!!!!Купил спонта...


### Возможно в обучающей выборке есть дубликаты строк. Проверим это.

In [73]:
len(texts) - train_data.drop_duplicates().shape[0]

115

In [74]:
train_data[train_data.duplicated()]

,label,text
139,1.0,Конечно увидев цену на телефоне немного как бы...
148,0.0,Сам телефон минус! Критически мало оперативн...
149,0.0,"Попал мне случайно (на халяву), пользуюсь год...."
313,1.0,После 6 месяцев работает как новый
494,1.0,Отличный телефон за свои деньги
501,0.0,В Sony Xperia XZ много хорошего (смотрите спис...
513,1.0,Покупать «сёмерку» вместо флагмана на Android ...
574,1.0,Безрамочный экран выглядит эффектно Красивый ...
607,1.0,хорошие
645,1.0,Супер


### Удалим повторы.

In [75]:
train_data.drop_duplicates(inplace = True)

### Проверим наличие отзывов с одинаковым текстом, но с разными оценками.

In [76]:
train_data[train_data.duplicated('text', keep=False)]

,label,text
1002,1.0,Отличный телефон!
1116,1.0,отличный телефон
2066,0.0,Отличный смартфон!
2473,0.0,Отличный телефон!
2503,0.0,отличный телефон
7492,1.0,
7982,0.0,
13654,1.0,Отличный смартфон!


### Удалим повторы.

In [77]:
train_data.drop_duplicates('text', keep=False, inplace = True)

### Посмотрим на отзывы, состоящие из одного слова.

In [78]:
d = {item: idx  for idx, item in enumerate(list(train_data.text)) if len(item.split())==1 }
for key, value in d.items():
    print(str(key) + ' ' + str(value))

https://irecommend.ru/content/samsung-galaxy-s8-plus-bezgranichnyi-ekran      8061
Лучший      9661
классно      7243
5-      1780
Супер!!!      1019
ОК      3912
Разочарование...      2610
пользовался             1925
Доволен.       628
Отличные       3556
хорошие      206
СУПЕР!!!!      8030
Крутой      10632
неплохое.      779
Отлично!      1709
Тормоз             184
Хорошие!      7228
Мощный      12767
Трудный      794
вау      663
Хорош      10046
Отличное       987
Н             13669
---      704
шикарно      7633
Лучший!       9865
Хорошее      245
Среднее.      248
хорошие  2467
доволен.      272
Очень&#8201;хотела&#8201;купить&#8201;именно&#8201;такой&#8201;телефон.&#8201;Последняя&#8201;версия&#8201;Андроид&#8201;4.4,&#8201;простая&#8201;в&#8201;управлении.&#8201;На&#8201;&#8201;крышке&#8201;сбоку&#8201;есть&#8201;переключатель&#8201;sim,&#8201;что&#8201;очень&#8201;удобно.&#8201;Снимала&#8201;смартфоном&#8201;детский&#8201;праздник,&#8201;снимки&#8201;получились&#8201;отли

### Заметим, что в тестовой выборке отзывы развёрнутые, значит, скорее всего, в обучении нам не понадобятся отзывы из одного слова.  Удалим эти отзывы.

In [79]:
train_data.drop(train_data.index[list(d.values())], inplace=True)

### Посмотрим на баланс классов тональности отзывов.

In [83]:
train_data.label.describe()

count    15619.000000
mean         0.889814
std          0.313132
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: label, dtype: float64

In [94]:
print('Количество положительных отзывов выборки составляет {}% размера всей выборки.'.format
      (round(float(sum(train_data.label)) / len(train_data.label) * 100, 2)))

Количество положительных отзывов выборки составляет 88.98% размера всей выборки.


In [97]:
print('Число отзывов меньшего класса: {}'.format(
    min(train_data[train_data.label==1].shape[0],train_data[train_data.label==0].shape[0])))

Число отзывов меньшего класса: 1721


### Выборка не является сбалансированной. Выполним undersampling и получим обучающую выборку с одинаковым количеством положительных и отрицательных отзывов.

In [99]:
train_data = train_data.groupby('label').apply(
    lambda x: x.sample(n=min(train_data[train_data.label==1].shape[0], train_data[train_data.label==0].shape[0])))

In [102]:
print('Размер обучающей выборки: {}'.format(train_data.shape[0]))

Размер обучающей выборки: 3442


In [103]:
texts = list(train_data.text)
labels = list(train_data.label)

### Построение моделей классификации.

In [104]:
vectorizers = [TfidfVectorizer, CountVectorizer]
classifiers = [LogisticRegression, LinearSVC]

for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v()), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.7629322548028312

LogisticRegression CountVectorizer
0.7460734748904618

LinearSVC TfidfVectorizer
0.7475286484664643

LinearSVC CountVectorizer
0.7068697337377824



### Добавим в модели биграммы.

In [105]:
for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v(ngram_range=(1, 2))), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.7640942028985508

LogisticRegression CountVectorizer
0.7597312099764071

LinearSVC TfidfVectorizer
0.7684546680148299

LinearSVC CountVectorizer
0.7240057296932928



### Добавляем в модель стоп-слова.

In [106]:
for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v(ngram_range=(1, 2), stop_words=stopwords.words('russian'))), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.7632195820694305

LogisticRegression CountVectorizer
0.7513094034378159

LinearSVC TfidfVectorizer
0.7632229524772497

LinearSVC CountVectorizer
0.7138388945062352



### Попробуем добавить в модель буквенные n-граммы.

In [107]:
for c in classifiers:
    for v in vectorizers:
        model = Pipeline([("vectorizer", v(analyzer='char_wb', ngram_range=(3, 5), stop_words=stopwords.words('russian'))), ("classifier", c())])
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.77600522413212

LogisticRegression CountVectorizer
0.7425935288169868

LinearSVC TfidfVectorizer
0.7728159757330637

LinearSVC CountVectorizer
0.7106370070778565



Считывание тестовых данных

In [108]:
test_parser = bs4.BeautifulSoup(test_data, 'lxml')
test_review_list = test_parser.findAll('review')
test_texts = []
for i in test_review_list:
    test_texts.append(i.text)

In [109]:
def n_p_func(l):
    r = []
    for i in l:
        if i == 1:
            r.append('pos')
        else:
            r.append('neg')
    return r

In [110]:
def write_res(model):
    model.fit(texts, labels)
    res = model.predict(list(test_texts))

    id_list = [i for i in range(100)]
    d = {'Id': id_list, 'y': n_p_func(res)}
    data_out = pd.DataFrame(d)

    data_out.to_csv('6_week_res', index=False)

### Лучшее значение accuracy было получено с использованием LogisticRegression с TfidfVectorizer. Примерно такой же результат был получен с использованием LinearSVC с TfidfVectorizer. 

In [111]:
model = Pipeline([("vectorizer", TfidfVectorizer(ngram_range=(1, 2))), ("classifier", LinearSVC())])
write_res(model)

In [117]:
model = Pipeline([("vectorizer", TfidfVectorizer(ngram_range=(1, 2))), ("classifier", LogisticRegression())])
write_res(model)

## Попробуем доработать модель с помощью более тщательной предобработки текстов

In [118]:
texts = list(train_data.text)

### Удалим из текста буквы английского алфавита, цифры и знаки пунктуации

In [119]:
my_punct = r'"#$%&\'()*,/:;<=>@[\\]^_`{|}~'

In [120]:
for i in range(len(texts)):
    for j in (string.digits + string.ascii_letters + my_punct):
        texts[i] = texts[i].replace(j, ' ')
    texts[i] = ' '.join(texts[i].split())

In [121]:
for c in classifiers:
    for v in vectorizers:
        print(str(c().__class__.__name__) + ' ' + str(v().__class__.__name__))
        model = Pipeline([("vectorizer", v(min_df= 2, ngram_range=(1, 2), stop_words=stopwords.words('russian'))), ("classifier", c())])
        print(cross_val_score(model, texts, labels, scoring='accuracy', cv = 5).mean())
        print

LogisticRegression TfidfVectorizer
0.7681605999325918

LogisticRegression CountVectorizer
0.739103471520054

LinearSVC TfidfVectorizer
0.7449182676103808

LinearSVC CountVectorizer
0.7045331985170206



In [126]:
clf = Pipeline([('vect', TfidfVectorizer()), ('classifier', LinearSVC())])
clf.get_params()

{'classifier': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': True,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__loss': 'squared_hinge',
 'classifier__max_iter': 1000,
 'classifier__multi_class': 'ovr',
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
           dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, s

In [127]:
parameters = {
    'vect__analyzer': ['word', 'char', 'char_wb'],
    'vect__min_df' : np.arange(1, 10),
    'vect__binary' : [True, False],
    'vect__norm' : ['l2', 'l1'],
    'vect__stop_words' : [None, stopwords.words('russian')],
    'vect__ngram_range' : [(1, 1), (1, 2), (1, 3), (3, 5), (3, 6), (3, 7), (2, 9), (3, 8), (3, 9)]
}

In [128]:
grid_cv = RandomizedSearchCV(clf, parameters, n_iter = 100, scoring = 'accuracy')

In [129]:
%%time
grid_cv.fit(texts, labels)

Wall time: 43min 37s


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'vect__analyzer': ['word', 'char', 'char_wb'], 'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (3, 5), (3, 6), (3, 7), (2, 9), (3, 8), (3, 9)], 'vect__binary': [True, False], 'vect__min_df': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'vect__stop_words': [None, [u'\u0438', u'\u0432', u'\u...043d\u043e', u'\u0432\u0441\u044e', u'\u043c\u0435\u0436\u0434\u0443']], 'vect__norm': ['l2', 'l1']},
          pre_dispatch='

In [130]:
print(grid_cv.best_params_)

{'vect__ngram_range': (3, 8), 'vect__analyzer': 'char', 'vect__binary': False, 'vect__min_df': 2, 'vect__stop_words': None, 'vect__norm': 'l2'}


In [131]:
print(grid_cv.best_score_)

0.7733875653689716


In [132]:
clf_grid = grid_cv.best_estimator_

In [133]:
write_res(clf_grid)

### С помощью подбора параметров удалось добиться улучшения качества модели.

### Интерактивная демонстрация модели

In [152]:
import pickle

In [153]:
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2))
X = tfidf_vect.fit_transform(texts)

In [154]:
clf = LogisticRegression()
clf.fit(X, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [155]:
with open('picle_classificator.pcl', 'wb') as fileout:
    pickle.dump(clf, fileout)

In [156]:
with open('picle_vectorizer.pcl', 'wb') as fileout:
    pickle.dump(clf, fileout)

### Выводы: 
### 1) В ходе исследования мы построили модель, предсказывающую тональность отзывов на телефон из тестовой выборки с точностью 94%.
### 2) Для достижения лучшего качества модели нужно улучшить токенизацию текста, составить словарь стоп-слов вручную и увеличить обучающую выборку.
### 3) Вместо подхода bag-of-words можно попробовать использовать другой подход - word2vec.